In [1]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import os
from sqlalchemy import delete,insert
from sqlalchemy.orm import sessionmaker

import json
import numpy as np
import pandas as pd
from datetime import date,datetime,timedelta

from sqla_schema import *

import ingest

data_directory = 'C:\\Users\\yoni.browning\\Documents\\DataJoint\\AllenData'
manifest_path = os.path.join(data_directory, "manifest.json")

def query_to_df(Q):
    df = pd.read_sql(Q.statement, Q.session.bind)
    return df

def spike_count(start_trigger,end_trigger,spike_ts):
    count = [None]*len(start_trigger)
    for ii,trigger in enumerate(start_trigger):
        count[ii] = np.sum(np.logical_and(spike_ts>=trigger,spike_ts<end_trigger[ii]))
    return count

In [2]:
engine = ingest.connect_to_db()
S = sessionmaker(engine)()
#Base.metadata.drop_all(engine, tables=(TrialSpikeCount.__table__,))
#Base.metadata.create_all(engine, tables=(TrialSpikeCount.__table__,))



In [ ]:
# Get the unique sessions for which we have spiketimes
unique_sessions = query_to_df(S.query(Session.id).\
            join(SessionProbe).\
            join(Channel).\
            join(Unit).\
            join(UnitSpikeTimes,Unit.id==UnitSpikeTimes.unit_id).group_by(Session.id)).values
unique_sessions

In [ ]:
def spike_count(start_trigger,end_trigger,spike_ts):
    count = [None]*len(start_trigger)
    for ii,trigger in enumerate(start_trigger):
        count[ii] = np.sum(np.logical_and(spike_ts>=trigger,spike_ts<end_trigger[ii]))
    return count

for ii,this_session in enumerate(unique_sessions):
    print('Session = ' + str(this_session[0]))
    stim_table = query_to_df(S.query(Session.id.label('session_id'),\
            StimulusPresentation.id.label("stimulus_id"),\
            StimulusPresentation.stop_time,\
            StimulusPresentation.start_time).\
            join(StimulusPresentation).filter(Session.id==int(this_session[0])))

    unit_table =  query_to_df(S.query(Session.id.label('session_id'),\
            Unit.id.label("unit_id"),\
            UnitSpikeTimes.spike_times).join(SessionProbe).\
            join(Channel).join(Unit).\
            join(UnitSpikeTimes,Unit.id==UnitSpikeTimes.unit_id).filter(Session.id==int(this_session[0])))


    duration = stim_table.stop_time-stim_table.start_time

    for ii,row in unit_table.iterrows():
        print('Unit  = ' + str(row.unit_id) + ' is ' +  str(ii) + ' of ' + str(len(unit_table)) ) 
        count = spike_count(stim_table.start_time,stim_table.stop_time,np.array(row.spike_times))
        this_df = pd.DataFrame(data = \
                               {'unit_id':int(row.unit_id),\
                                'stimulus_id':np.array(stim_table.stimulus_id.values).astype(int),\
                                'spike_count':count,\
                                'spike_rate':np.divide(count,duration)})
        this_df.to_sql('trial_spike_count', engine,index = False, if_exists='append')
    S.commit()

In [ ]:
query_to_df(S.query(TrialSpikeCount.spike_count,StimulusPresentation.stimulus_type_id).\
            join(StimulusPresentation).\
            filter(TrialSpikeCount.unit_id ==950907205 ))

In [ ]:
query_to_df(S.query(Session.id,Structure.name).\
            join(SessionProbe).join(Channel).join(Structure).group_by(Session.id,Structure.name))

In [ ]:
query_to_df(S.query(Session.id,Structure.name).\
                              join(SessionProbe).\
                              join(Channel).\
                              join(Structure).\
                              filter(Session.id==715093703).\
                              group_by(Session.id,Structure.name))


In [ ]:
BS = S.query(Structure).filter(Structure.name == "Field CA1")

In [ ]:
query_to_df(S.query(Session.id,UnitSpikeTimes.unit_id).\
    join(SessionProbe).\
    join(Channel).\
    join(Structure).\
    join(Unit).\
    join(UnitSpikeTimes,Unit.id==UnitSpikeTimes.unit_id).\
            filter(Session.id==int(715093703)).\
            filter(Structure.name == "Field CA1"))

In [ ]:
S.query(UnitSpikeTimes.spike_times).\
       filter(UnitSpikeTimes.unit_id==950911195).all()

In [ ]:
query_to_df(S.query(Session.id,TrialSpikeCount.unit_id).\
    join(StimulusPresentation,Session.id ==StimulusPresentation.session_id).\
    join(TrialSpikeCount,StimulusPresentation.id == TrialSpikeCount.stimulus_id).\
            group_by(Session.id,TrialSpikeCount.unit_id))

In [26]:
data = query_to_df(S.query(TrialSpikeCount.spike_rate,\
        StimulusPresentation.orientation,\
        StimulusPresentation.spatial_frequency).\
        join(StimulusPresentation).join(StimulusType).\
        filter(TrialSpikeCount.unit_id == 950907205).
        filter(StimulusType.name=='static_gratings'))

In [27]:
data = data.groupby(['spatial_frequency','orientation']).mean()

In [40]:
data.index.get_level_values(0)

Float64Index([0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.04, 0.04, 0.04, 0.04, 0.04,
              0.04, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.16, 0.16, 0.16, 0.16,
              0.16, 0.16, 0.32, 0.32, 0.32, 0.32, 0.32, 0.32],
             dtype='float64', name='spatial_frequency')

In [41]:
data = query_to_df(S.query(TrialSpikeCount.spike_rate,\
        StimulusPresentation.orientation,\
        StimulusPresentation.spatial_frequency).\
        join(StimulusPresentation).join(StimulusType).\
        filter(TrialSpikeCount.unit_id == 950907205).
        filter(StimulusType.name=='drifting_gratings'));

In [42]:
StimulusPresentation.temporal_frequency

,spike_rate,orientation,spatial_frequency
0,2.997392,225.0,0.04
1,5.995054,270.0,0.04
2,12.989219,45.0,0.04
3,13.988110,270.0,0.04
4,4.995828,270.0,0.04
...,...,...,...
623,24.979142,45.0,0.04
624,11.989988,0.0,0.04
625,25.478598,0.0,0.04
626,17.485400,315.0,0.04


In [53]:
unit_df = query_to_df(S.query(Session.id,TrialSpikeCount.unit_id).\
                join(SessionProbe).\
                join(Channel).\
                join(Structure).\
                join(Unit).\
                join(TrialSpikeCount,Unit.id==TrialSpikeCount.unit_id).group_by(Session.id,TrialSpikeCount.unit_id))


In [54]:
unit_df

,id,unit_id
0,721123822,950907205
1,721123822,950907209
2,721123822,950907216
3,721123822,950907220
4,721123822,950907222
5,721123822,950907225
6,721123822,950907229
7,721123822,950907233
8,721123822,950907235
9,721123822,950907242
